# SK Memory - RAG Pattern Primer

Learning objectives:

- Setting up SK to save memories
- Saving, retrieving, recalling, and deleting memories

Reference:
- https://github.com/Azure-Samples/azure-search-openai-demo
- https://github.com/azure-samples/semantic-kernel-rag-chat

**Important:**

- A memory includes at a minium a key, some text (a text chunk), and the embedding
- The embedding is used to perform a semantic search, but the text is what is used for augmentation

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.19.230804.2-preview"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System.ComponentModel;
using System.Net.Http;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Skills.Core;
using dotenv.net;

#!import Utils/ConsoleLogger.cs

const string MemoryCollectionName = "aboutMe";

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Get a Kernel with an embedding service

In [ ]:
var kernel = Kernel.Builder
            .WithLogger(ConsoleLogger.Log)
            .WithAzureChatCompletionService(deploymentName, endpoint, apiKey)
            .WithAzureTextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey)
            .WithMemoryStorage(new VolatileMemoryStore())
            .Build();

## Save memories - Method 1

In [ ]:
// ========= Store memories using the kernel =========

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andre");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I work as a tourist operator");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I've been living in Seattle since 2005");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");

## Save memories - Method 2 - using a skill

In [ ]:
// ========= Store memories using semantic function =========

// Add Memory as a skill for other functions
var memorySkill = new TextMemorySkill(kernel.Memory);
kernel.ImportSkill(memorySkill);

// Build a semantic function that saves info to memory
const string SaveFunctionDefinition = "{{save $info}}";
var memorySaver = kernel.CreateSemanticFunction(SaveFunctionDefinition);

var context = kernel.CreateNewContext();
context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.KeyParam] = "info5";
context["info"] = "My family is from New York";
await memorySaver.InvokeAsync(context);

## Retrive a memory by Key

In [ ]:
// ========= Test memory remember =========
Console.WriteLine("========= Example: Recalling a Memory =========");

var answer = await memorySkill.RetrieveAsync(MemoryCollectionName, "info1", logger: context.Log);
Console.WriteLine("Memory associated with 'info1': {0}", answer);

## Recall a memory based on relevance and a count limit

In [ ]:
Console.WriteLine("========= Example: Recalling an Idea =========");

answer = await memorySkill.RecallAsync("where did I grow up?", MemoryCollectionName, relevance: null, limit: 2, logger: context.Log);
Console.WriteLine("Ask: where did I grow up?");
Console.WriteLine("Answer:\n{0}", answer);

answer = await memorySkill.RecallAsync("where do I live?", MemoryCollectionName, relevance: null, limit: 2, logger: context.Log);
Console.WriteLine("Ask: where do I live?");
Console.WriteLine("Answer:\n{0}", answer);

## Recall a memory using an SK function

In [ ]:
Console.WriteLine("========= Example: Using Recall in a Semantic Function =========");

// Build a semantic function that uses memory to find facts
const string RecallFunctionDefinition = @"
Consider only the facts below when answering questions.

About me: {{recall 'where did I grow up?'}}
About me: {{recall 'where do I live?'}}

Question: {{$input}}

Answer:
";

var aboutMeOracle = kernel.CreateSemanticFunction(RecallFunctionDefinition, maxTokens: 100);
var result = await kernel.RunAsync(aboutMeOracle, new("Do I live in the same town where I grew up?")
{
    [TextMemorySkill.CollectionParam] = MemoryCollectionName,
    [TextMemorySkill.RelevanceParam] = "0.8"
});

Console.WriteLine("Do I live in the same town where I grew up?\n");
Console.WriteLine(result);

## Remove a memory by key

In [ ]:
// ========= Remove a memory =========
Console.WriteLine("========= Example: Forgetting a Memory =========");

result = await kernel.RunAsync(aboutMeOracle, new("Tell me a bit about myself")
{
    ["fact1"] = "What is my name?",
    ["fact2"] = "What do I do for a living?",
    [TextMemorySkill.RelevanceParam] = ".75"
});

Console.WriteLine("Tell me a bit about myself\n");
Console.WriteLine(result);

await memorySkill.RemoveAsync(MemoryCollectionName, "info1", logger: null);
result = await kernel.RunAsync(aboutMeOracle, new("Tell me a bit about myself"));

Console.WriteLine("Tell me a bit about myself\n");
Console.WriteLine(result);
